In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Project

In [2]:
!pip install Sentencepiece
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
import re
import random
import torch
from tqdm.notebook import tqdm
import transformers
from transformers import BertTokenizer
import re

In [4]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [5]:

tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-poem")

In [6]:

with open('/content/drive/MyDrive/alchemy',encoding = 'utf8') as f:
  text = f.read().replace('。', '。\n').split('\n')

text = [re.sub('\n', '', i) for i in text]
text = [re.sub(r'\*', '', i) for i in text]
text = [re.sub(r'●', '', i) for i in text]
text = [re.sub(r'\[','', i) for i in text]
text = [re.sub(r'\]', '', i) for i in text]
text = [re.sub(r'《 ', '', i) for i in text]
text = [i.strip() for i in text]

print(text[1:10])

['', '乾坤者，易之門戶，眾卦之父母。', '坎離匡廓，運轂正軸，牝牡四卦，以為橐籥。', '覆冒陰陽之道，尤工御者準繩墨，執銜轡，正規距，隨軌轍，處中以制外，數在律歷紀。', '月節有五六，經緯奉日使，兼並為六十，剛柔有表裏。', '朔旦屯直事，至暮蒙當受，晝夜各一卦用之依次序。', '即未至晦爽，終則復更始，日月為期度，動靜有早晚。', '春夏據內體，從子到辰巳，秋冬當外用，自午訖戌亥。', '賞罰應春秋，昏明順寒暑，爻辭有仁義，隨時發喜怒，如是應四時，五行得其理。']


In [7]:

tokens = tokenizer.encode(text, add_special_tokens= True)
tokens = np.array(tokens)
print(len(tokens))
tokens[1]

8184


100

# Making train and test 



In [8]:
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))
print(train[:10])

8184 6540 1635
[100 100 100 100 100 100 100 100 100 100]


## Creating 2 models for fine tuning: model_init и model

In [9]:
from transformers import GPT2LMHeadModel, AdamW

model_init = GPT2LMHeadModel.from_pretrained(
    "uer/gpt2-chinese-poem",
    output_attentions = False,
    output_hidden_states = False,
)


model = GPT2LMHeadModel.from_pretrained(
    "uer/gpt2-chinese-poem",
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

## Creating batch size,max_len and epochs

In [10]:
batch_size = 8 # number of sequenses
max_len = 64 #length of sequense
epochs = 10

n_train = len(train)//(batch_size*max_len)  
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

12 3


## Задаем оптимизатор, общее количество шагов 

In [11]:
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py is 0
                                            num_training_steps = total_steps)
"""
Warm up steps is a parameter which is used to lower the learning rate in order to reduce the impact of 
deviating the model from learning on sudden new data set exposure.
"""

def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## creating function for 


In [12]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids
 


## дообучаем модель, выводя accuracy для каждой эпохи 

In [13]:
print(train[6000])

100


In [14]:
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                            #  attention_mask=batch_mask,
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)

epoch 1/10 : training


  0%|          | 0/12 [00:00<?, ?it/s]

epoch 1/10 : validation


  0%|          | 0/3 [00:00<?, ?it/s]

epoch 2/10 : training


...

  0%|          | 0/3 [00:00<?, ?it/s]

epoch 10/10 : training


  0%|          | 0/12 [00:00<?, ?it/s]

epoch 10/10 : validation


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# Model without training
import textwrap

# prompt = "print('Hello world!') "
prompt = '朱熹'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=1,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


[CLS] 朱 熹 [SEP] 役 不 停 轨 ， 仲 夏 园 林 尽 绿 阴 。 一 雨 一 风 皆 有 意 ， 百 忧 千 虑 本 无 心 。 [SEP] 求 大 隐 聊 中 隐 ， 何 必 东 山 始 盍 簪 。 莫 把 升 沉 翻
手 记 ， 昔 年 轩 冕 是 浮 沉 。 [SEP] 为 浮 薄 误 京 华 ， 几 度 春 归 不 见 家 。 今 日 独 来 关 内 事 ， 数 株 临 水 为 桃 花 。 [SEP] 造 乾 元 开 造 化 ， 试 凭 风 月 洗
疵 瑕 。 万 方 玉 烛 调 元 气 ， 散 作 人 间 第 一 花 。 万 千 家 和 羹 鼎 ， 调 羹 心 愿 许 谁 加 惓 。 [SEP] ， 一 念 念
------------------
[CLS] 朱 熹 [SEP] 役 几 时 休 ， 又 向 天 涯 作 远 游 。 风 景 不 殊 人 自 老 ， 江 山 如 此 我 何 求 。 一 年 秋 色 黄 花 酒 ， 万 里 西 风 白 雁 楼 。 回 首 故 园 归 未 得
， 梦 魂 夜 夜 绕 松 楸 。 [SEP] 阳 城 下 水 悠 悠 ， 送 尽 行 人 古 渡 头 。 两 岸 青 山 看 不 尽 ， 一 川 明 月 去 难 留 。 梦 魂 每 绕 关 山 路 ， 书 信 频 通 淮 海 秋 。 此
去 定 知 逢 圣 代 ， 好 将 民 隐 达 宸 旒 。 [SEP] ， 为 国 活 苍 生 赤 子 孙 ， 莫 使 民 忧 民 愚 民 愚 。 [SEP]
------------------
[CLS] 朱 熹 [SEP] 役 两 旬 馀 ， 不 雨 经 时 气 若 嘘 。 应 为 向 人 忧 旱 魃 ， 故 将 祈 福 报 神 鱼 。 [SEP] 前 月 色 明 如 昼 ， 檐 后 风 声 吼 似 车 。 病 体 幸 宽 心
未 稳 ， 且 偷 闲 暇 读 残 书 。 [SEP] 祷 何 曾 效 祝 田 ， 愿 随 飞 雨 下 山 川 。 油 然 已 慰 三 农 望 ， 旱 甚 还 祈 大 有 年 。 濒 海 土 田 成 沃 壤 ， 沿 街 禾 黍 秀 平 川
。 老 农 甚 喜 闻 谣 颂 ， 不 愧 穹 碑 万 口 传 。 [SEP] ， 歌 咏 我 惭 不 才 悭 成 句 不 如 泉 拟 ， 聊 当 挹 清 

In [16]:
import textwrap

prompt = '朱熹'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=10,
    do_sample=True,
    temperature=5.,
    top_k=50,
    top_p=0.15,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


[CLS] 朱 熹 [SEP] 役 岁 何 穷 ， 万 物 均 含 造 化 功 。 天 道 本 无 盈 损 有 ， 世 情 徒 有 异 常 风 。 三 千 路 绝 云 泥 隔 ， 九 十 风 高 雪 霰 蒙 。 会 得 圣 贤 心 术 妙
， 莫 将 消 息 问 穷 通 。 [SEP] 人 无 计 遣 花 知 ， 只 祷 芳 菲 便 及 时 。 若 道 根 株 能 孕 秀 ， 何 妨 桃 李 自 开 迟 。 人 心 有 待 终 归 厚 ， 物 理 无 私 不 可 推 。 开
到 春 风 终 始 盛 ， 天 心 人 事 每 相 疑 。 须 知 时 令 人 道 理 最 说 难 为 人 情 由 ， 岂 偏 。 莫 守 。
------------------
[CLS] 朱 熹 [SEP] 驭 日 如 年 ， 圣 主 忧 民 一 念 虔 。 恭 默 帝 心 逾 二 九 ， 讴 歌 国 祚 启 三 千 。 星 光 宝 炬 辉 瑶 席 ， 月 彩 虹 桥 丽 采 椽 。 一 德 格 天 覃 广 运
， 垂 衣 永 叶 太 平 篇 。 [SEP] 殿 晨 开 露 未 乾 ， 诏 书 春 燕 锡 龙 鸾 。 云 随 黄 道 天 开 辟 ， 日 丽 雕 舆 地 极 寒 。 五 色 祥 烟 迎 剑 佩 ， 九 重 佳 气 绕 衣 冠 。 小
臣 何 幸 承 清 问 ， 愿 效 封 人 祝 寿 看 。 圣 寿 比 干 封 词 华 夷 颂 声 乾 乾 嘉 事 美 并 储 祥 ， 万 年
------------------
[CLS] 朱 熹 [SEP] 驭 日 ， 大 火 井 坎 涂 。 西 北 有 星 晷 ， 东 南 多 地 枢 。 天 弧 不 能 挂 ， 河 鼓 难 为 输 。 岂 不 怀 安 乐 ， 所 思 在 远 途 。 [SEP] 我 虞 夏 君
， 万 国 同 车 书 。 周 家 历 五 纬 ， 皇 极 次 北 隅 。 四 时 各 有 朔 ， 七 职 乃 其 馀 。 大 位 在 君 子 ， 小 德 在 唐 虞 。 君 臣 以 敬 德 ， 天 地 以 为 徒 。 万 古 此 明 月
， 百 世 之 冰 壶 。 谁 知 一 年 内 ， 光 彩 楼 台 榭 上 ， 清 光 不 动 摇 帝 极 ， 日 除 元 精 凝 为 最 上
------------------
[CLS] 朱

In [17]:
def generate(prompt, len_gen=20, temperature=5):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model(context, past_key_values=past).values()
        # token = torch.argmax(output[..., -1, :], dim=-1)
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    return sequence

In [18]:
import textwrap



prompt = '朱熹'
#translator  = Translator(from_lang = 'zh',to_lang = 'ru')
x = (textwrap.fill(generate(prompt, 200, temperature=1.), 120))

print(x)


  0%|          | 0/200 [00:00<?, ?it/s]

[CLS] 朱 熹 [SEP] 驭 咸 淳 盛 ， 王 寿 曾 有 国 寿 筹 。 蠲 租 叩 阙 尽 纯 嘏 ， 锡 福 觊 灵 频 接 福 。 君 为 嘉 谋 相 庆 积 ， 廷 尉 上 最 孰 与 侔 。 宗 社 本 朝 学 颇 牧
， 便 见 黄 阁 相 书 求 。 君 家 此 中 修 政 事 ， 喜 溢 一 室 方 小 休 。 有 才 毕 力 能 四 体 ， 俗 料 安 逸 能 无 愁 。 邑 居 端 可 就 乡 闬 ， 一 身 苟 得 安 林 丘 。 肯 来 左
右 坐 经 月 ， 旁 有 琴 鹤 时 相 求 。 厨 传 将 苏 牛 酒 馔 ， 足 欢 无 忧 羞 肴 赋 酒 。 从 头 倾 耳 润 泽 ， 创 纷 咨 休 养 拙 帑 罄 ， 惟 勤 悴 忧 患 求 赫 全 收 鸠 [UNK] 得
谢 不 休 。 圣 神 休 。 世 有 喜 志 就 有 嗣 ， 琅 复 休 。 善 为 君 德 风 猷 。 三 杨 李


# Creating translator

In [19]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [20]:
tokenizer_1 = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en",use_fast = False)
model_1 = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
text = x
tokenized_text_1 = tokenizer_1.prepare_seq2seq_batch([text], return_tensors='pt')#,paddind = True)
translation_1 = model_1.generate(**tokenized_text_1)#(src_text, return_tensors="pt", padding=True))
translated_text_1 = tokenizer_1.batch_decode(translation_1, skip_special_tokens=False)[0]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3478: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [21]:
print(translated_text_1)

<pad> [CLS] [SEP] The King’s life has been set up for the good of the land. The King’s house has built the pure and pure state of the land, and the Sifu spirits have been blessed. The King has made the best effort for the sake of the people, the best of the king’s captains and the best of the country’s leaders.


In [22]:
tokenizer_2 = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en",use_fast = False)
model_2 = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
text_2 = x
input_ids = tokenizer_2(text, return_tensors="pt").input_ids
outputs = model_2.generate(input_ids=input_ids,temperature = 1)
print("Generated:", tokenizer_2.batch_decode(outputs, skip_special_tokens=True)[0])

Generated: [CLS] [SEP] The King’s life has been set up for the good of the land. The King’s house has built the pure and pure state of the land, and the Sifu spirits have been blessed. The King has made the best effort for the sake of the people, the best of the king’s captains and the best of the country’s leaders.
